In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-13 14:48:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.23MB/s    in 0.8s    

2021-11-13 14:48:35 (1.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [8]:
# filter DataFrame for total_votes above or equal to 20
df1 = vine_df.filter(vine_df.total_votes >= 20)
df1.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [10]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
df2 = df1.filter((df1.helpful_votes / df1.total_votes) >= 0.5)
df2.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [11]:
# Create paid vine DataFrame
paid_df = df2.filter(df2.vine == 'Y')
paid_df.show(20)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [13]:
# Create unpaid vine DataFrame
unpaid_df = df2.filter(df2.vine == 'N')
unpaid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [14]:
# paid total number of reviews
total_paid_reviews = paid_df.count()
total_paid_reviews

0

In [15]:
# paid 5-star reviews
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_reviews

0

In [17]:
# paid 5-star reviews percentage
#paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
#paid_five_star_percent
#error because division by zero is not possible

In [18]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

403807

In [19]:
# unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star_reviews

242889

In [20]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

60.14977452099642